In [12]:
from datetime import date
import datetime
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import pyodbc
import sqlalchemy as sa
import urllib
import re

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.action_chains import ActionChains

cur_day = date.today().strftime("%m/%d/%y")

params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=DOITSQL2017.illinois.gov;"
                                 "DATABASE=CMS_DAP;"
                                 "UID=cms_dap_owner;"
                                 "PWD=IL@veFriday#8102")

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params),
                         fast_executemany=True)

login_name = ''
password = ''

file_ls = []

In [13]:
def partial_link_helper(ltext, timeout=10):
    """Finds link in the page with given text and clicks it 
    
    Parameters:
        ltext: link text
        timeout: time to wait until the link is found in seconds

    Raise: 
        TimeoutException: when link not found on page
    """
    try:
        print(ltext)
        element_present = EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, ltext))
        WebDriverWait(driver, timeout).until(element_present)
        time.sleep(0.5) # A sleep is after a wait, might be redundant
        driver.find_element_by_partial_link_text(ltext).click()
    except TimeoutException:
        raise("Timed out waiting for link to load")

def id_action(id_name, val, action_type):
    """Manipulates individual element of a webpage

    Parameters: 
        id_name: the name of an html element id on the webpage
        val(optional): typically '' but used to supply text to be entered into textbox or the option to be selected in dropdown 
        action_type: action to take on page
            'send_keys' -> enters text in val into textbox
            'click' -> clicks element
            'clear' -> clears textbox                    - NOT USED
            'select' -> selects text supplied in element - NOT USED
    """
    timeout = 10
    try:
        element_present = EC.presence_of_element_located((By.ID, id_name))
        WebDriverWait(driver, timeout).until(element_present)
        time.sleep(0.25)
        if action_type == 'send_keys':
            driver.find_element_by_id(id_name).send_keys(val)
        elif action_type == 'clear':
            driver.find_element_by_id(id_name).clear()
        else:
            element_clickable = EC.element_to_be_clickable((By.ID, id_name))
            WebDriverWait(driver, timeout).until(element_clickable)
            if action_type == 'select':
                select = Select(driver.find_element_by_id(id_name))
                select.select_by_visible_text(val)
            elif action_type == 'click':
                driver.find_element_by_id(id_name).click()
    except TimeoutException:
        print ("Timed out waiting for id_action")

def log_in():
    # Launches Chrome then Logs into bidbuy using the username and password supplied in setup
    # NOTES: currently the chrome executable is hardcoded as LaReyna's chrome executable in OneDrive. Not great practice will break
    global driver
    driver = webdriver.Chrome(executable_path=r"C:\Users\lareyna.griffin\OneDrive - State of Illinois - Dept. of Innovation & Technology\Desktop Analysis\chromedriver.exe")
    driver.get("https://www.bidbuy.illinois.gov/bso/view/login/login.xhtml")
    id_action('home-sign-in-btn', '', 'click')
    id_action('homeLoginForm:loginId', login_name, 'send_keys')
    id_action('homeLoginForm:password', password, 'send_keys')
    id_action('homeLoginForm:sign-in-dialog-btn', '', 'click')
    print('login completed')



def get_to_em_folder():
    # Once logged in this navigates to the folder that is associated with the user
    time.sleep(2)
    driver.get('https://www.bidbuy.illinois.gov/bso/reporting/adHocReports.sdo')
    time.sleep(10)
    #TODO What does this do? Doesn't need to be changed unless bidbuy changes. This also may be an error handling thing
    for i, b in enumerate(driver.find_elements_by_tag_name('button')):
        if b.text == 'Cancel':
            b.click()
    time.sleep(2)
    id_action('main_view', '', 'click')
    time.sleep(2)
    #Opens Repository in menu
    targ_lis = driver.find_element_by_id('menuList').find_elements_by_tag_name("li")
    for li in targ_lis:
        if li.text == 'Repository':
            li.click()
            break
    #check if folder already present
    time.sleep(5)
    targ_folders = driver.find_elements_by_tag_name('li')
    if 'ERIC.LANGOWSKI@ILLINOIS.GOV' not in [li.text for li in targ_folders]:
        #TODO Not sure what this does, but I think it adds a folder with the given username
        time.sleep(2)
        id_action('handler2', '', 'click')
        time.sleep(2)
        id_action('handler4', '', 'click')
        time.sleep(2)
        id_action('handler7', '', 'click')
        time.sleep(2)

    #Enters the folder labeled with the username
    targ_folders = driver.find_elements_by_tag_name('li')
    for li in targ_folders:
        if li.text == 'ERIC.LANGOWSKI@ILLINOIS.GOV':
            li.click()
            break
    print('get to em folder')

In [14]:
def record_file_name(ltext, fparam1, fparam2):
    file_ls.append([ltext, fparam1, fparam2])

def actually_export(ltext, fparam1, fparam2=''):
    """This navigate to click the export as CSV button and adds it to the list of files changed
    fparam1 & fparam2 are for FY files and only used to record
    """
    action = ActionChains(driver)
    time.sleep(10)
    firstLevelMenu = driver.find_element_by_id("export")
    action.move_to_element(firstLevelMenu).perform()
    time.sleep(1)
    element_present = EC.presence_of_element_located((By.XPATH, "//p[contains(text(),'As CSV')]"))
    WebDriverWait(driver, 60*35).until(element_present) #35 Minutes!
    time.sleep(2)
    driver.find_element_by_xpath("//p[contains(text(),'As CSV')]").click()
    time.sleep(2)
    while len(driver.window_handles) > 1:
        time.sleep(2)
    time.sleep(5)
    record_file_name(ltext, fparam1, fparam2)

def export_report(ltext):
    """Open link and if it's an FY report then download each years report with the two release numbers

    Paramters: 
        ltext: partial link text

    At the moment program will exit if a link is not found
    """
    time.sleep(10) #sleep would be better placed in upper layer loop
    partial_link_helper(ltext)
    time.sleep(10)

    element_present = EC.presence_of_element_located((By.ID, 'export'))
    WebDriverWait(driver, 60*15).until(element_present)
    if ltext[:3] == 'FY:':
        #change A.Fiscal Year
        fy_years = range(16,23) #TODO this is hard coded an not going to work in the future
        for i in fy_years:
            fltr_str = "//a[@title='" + str(i) + "']"
            driver.find_element_by_class_name('sSelect-input').click()
            time.sleep(5)
            driver.find_element_by_xpath(fltr_str).click()
            time.sleep(5)
            #change B.Release Nbr
            fltr_types = ['is not equal to', 'equals']
            for j in fltr_types:
                driver.find_elements_by_class_name('operator')[1].click()
                time.sleep(1)
                driver.find_element_by_xpath("//p[contains(text(),'" + j + "')]").click()
                time.sleep(2)
                #apply
                driver.find_element_by_xpath("//span[contains(text(),'Apply')]").click()
                time.sleep(15)
                print('~~' + str(i) + '--' + j + '~~')
                actually_export(ltext, str(i), j)
    else:
        actually_export(ltext, '')
    driver.back()
    time.sleep(5)

        
def export_kpi_reports(download_fy=False):
    """Downloads (almost) all reports in the current / get_em_to_folder folder - will likely silently fail if not the same

    Parameters:
        download_fy: bool -> if true will also download all the FY reports
    """
    time.sleep(2)
    cur_reports = driver.find_element_by_id('resultsList').find_elements_by_tag_name("a")
    to_download = [a.text for a in cur_reports if a.text[:4] == 'KPI:']
    if download_fy:
        to_download = [a.text for a in cur_reports if a.text[:3] == 'FY:']
        
    for targ_a in to_download:
        if targ_a == 'KPI: PO Domain CMSItems':
            pass
        else:
            time.sleep(10)
            cur_links = driver.find_element_by_id('resultsList').find_elements_by_tag_name("a")
            for a in cur_links:
                if a.text == targ_a:
                    ltext = a.text
                    try:
                        export_report(ltext)
                    except Exception as inst:
                        print('error~~~sleeping')
                        print(inst)
                        time.sleep(15*60) #This is 15 minutes!!
                        get_to_em_folder()
                        export_report(ltext)
                    break
    time.sleep(2*60) # I could be wrong, but this doesn't need to be here? maybe it's meant to be indented to break up the two 
    #Actually this might be to give time to download reports


In [15]:
def read_bid_buy(file_name):
    """Takes in a csv file and modifies it so that it can be entered into
    the sql table
    """
    tmp = pd.read_csv(file_name, dtype=str)
    tmp = tmp[(tmp.iloc[:, 0] != tmp.columns[0])]
    #Removes all rows that are missing or have blank PO numbers
    if 'PO Nbr' in tmp.columns:
        tmp = tmp[(tmp['PO Nbr'].notna()) & 
        (tmp['PO Nbr'] != '')]
    if 'Release Nbr' in tmp.columns:
        tmp['Release Nbr'] = tmp['Release Nbr'].str.split('.').str[0].str.replace(',', '')
    tmp = tmp.drop_duplicates()
    tmp.columns = tmp.columns.str.replace(' ','_')
    return tmp

def upload_to_sql(tmp, sql_name, append=False):
    """This takes pandas data and a sql table and either appends it or replaces it.
    Then reports the old and new column counts

    Parameters:
        tmp -> results from read_bid_buy, a pandas dataframe
        sql_name -> name of sql table in database
        append -> bool for either append or replace
    """
    if append:
        param = 'append'
    else:
        param = 'replace'
    try:
        rcount = str(pd.read_sql('select count(*) as cnt from ' + sql_name, engine).iloc[0, 0])
    except:
        rcount = 'DOES NOT EXIST'
    print('~~~' + sql_name + '~~~ (old rows:' + rcount + ')')
    print('(new rows: ' + str(tmp.shape[0]) + ')')
    tmp.to_sql(sql_name, engine, index=False, if_exists=param, chunksize = 100)
    rcount = str(pd.read_sql('select count(*) as cnt from ' + sql_name, engine).iloc[0, 0])
    print('(new rows sql: ' + rcount + ')')
    
def upload_bid_buy(file_ls):
    """This selects all the files in the given folder and uploads all files beginning with 'Report' and ending with '.csv'
        files are read and then uploaded to a sql server
        Then it deletes all of those files
    """
    dfold = 'C:/Users/lareyna.griffin/Downloads/' #TODO change to setup
    cur_files = [i for i in os.listdir(dfold) if i[:6] == 'Report' and i.endswith(".csv")] 
    # This is extremely likely to upload extra reports
    seen_reports = []

    for file in cur_files:
        file_time = datetime.datetime.now() - datetime.datetime.fromtimestamp(os.path.getctime(dfold + file))
        if file_time.days > 1:
            print('old file should be deleted')
            print(file)
            break
        file_num = re.findall(r"\((\d+)\)", file) #Selects all digits in () so text(24) -> 24
        if not file_num:
            file_num = 0
        else:
            file_num = file_num[0]
        report_type = file_ls[int(file_num)][0] #Warning: all extra files would be listed as file_ls[0] 

        print(file)
        print(report_type)
        if not os.stat(dfold + file).st_size == 0:
            tmp = read_bid_buy(dfold + file)
            sql_name = report_type.replace(' ', '').replace('KPI:', 'BidBuy_').replace('FY:', 'BidBuy_')
            tmp['modified_date'] = cur_day
            if report_type not in seen_reports: #drop old table and upload
                upload_to_sql(tmp, sql_name)
                seen_reports.append(report_type)
            elif report_type in ['FY: PO Items Table', 'FY: PO Items Full']: #append
                upload_to_sql(tmp, sql_name, append=True)
            os.remove(dfold + file) #This can delete unintended files


In [16]:
log_in()

login completed


In [17]:
get_to_em_folder()


get to em folder


In [8]:
export_kpi_reports(False)

KPI: All Documents Domain
KPI: PO Domain
KPI: PO Domain CMS
KPI: PO Domain CMS Items
KPI: Requisition Domain
KPI: Solicitations Domain Bids
KPI: Solicitations Domain Quotes
KPI: Vendor Domain


In [ ]:
upload_bid_buy(file_ls)

Report1654190463959 (1).csv
KPI: PO Domain
~~~BidBuy_PODomain~~~ (old rows:110897)
(new rows: 112884)
(new rows sql: 112884)
Report1654190463959 (2).csv
KPI: PO Domain CMS
~~~BidBuy_PODomainCMS~~~ (old rows:6694)
(new rows: 6925)
(new rows sql: 6925)
Report1654190463959 (3).csv
KPI: PO Domain CMS Items
~~~BidBuy_PODomainCMSItems~~~ (old rows:50600)
(new rows: 51452)
(new rows sql: 51452)
Report1654190463959 (4).csv
KPI: Requisition Domain
~~~BidBuy_RequisitionDomain~~~ (old rows:130592)
(new rows: 132726)
(new rows sql: 132726)
Report1654190463959 (5).csv
KPI: Solicitations Domain Bids
~~~BidBuy_SolicitationsDomainBids~~~ (old rows:28850)
(new rows: 29262)
(new rows sql: 29262)
Report1654190463959 (6).csv
KPI: Solicitations Domain Quotes
~~~BidBuy_SolicitationsDomainQuotes~~~ (old rows:53266)
(new rows: 54054)
(new rows sql: 54054)
Report1654190463959 (7).csv
KPI: Vendor Domain
~~~BidBuy_VendorDomain~~~ (old rows:24148)
(new rows: 24307)
(new rows sql: 24307)
Report1654190463959.csv
KP

In [11]:
from subprocess import Popen, PIPE

file_loc = "C:/Users/lareyna.griffin/OneDrive - State of Illinois - Dept. of Innovation & Technology/Desktop Analysis/Ad hoc/"

cmd = ["C:/Program Files/R/R-4.1.2/bin/RScript.exe",
       "boss_sharepoint_view.R", 
       file_loc + "boss_sharepoint_view.R"]

p = Popen(cmd, cwd=file_loc, stdin=PIPE, stdout=PIPE, stderr=PIPE)     
output, error = p.communicate()

# PRINT R CONSOLE OUTPUT (ERROR OR NOT)
if p.returncode == 0:            
  print('R OUTPUT:\n {0}'.format(output))            
else:                
  print('R ERROR:\n {0}'.format(error))

R OUTPUT:
 b'[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"\r\n[1] "old rows:"\r\n[1] 1488\r\n[1] "new rows:"\r\n[1] 1488\r\n# Source:   table<BidBuy_Consolidated_PODomain_CMSItems> [?? x 34]\r\n# Database: Microsoft SQL Server\r\n#   14.00.3436[CMS_DAP_Owner@IL084SQL2017B/CMS_DAP]\r\n   PO_Nbr  Fiscal_Year Alternate_Id Old_Contract_Nu~ Short_Descripti~ Begin_Date\r\n   <chr>   <chr>       <chr>        <chr>            <chr>            <chr>     \r\n 1 17-416~ 17          9100000370   <NA>             MC Continuing L~ 01/26/2017\r\n 2 17-416~ 17          9100000916   <NA>             Large Format Sc~ 07/01/2017\r\n 3 17-416~ 17          9100000098   <NA>             MC Fleet Fuel C~ 06/01/2016\r\n 4 17-416~ 17          9100000361   <NA>             JPMC PAPER AND ~ 03/06/2017\r\n 5 18-416~ 18          <NA>         <NA>             Healthcare Cons~ 05/01/2013\r\n 6 18-416~ 18          <NA>         <NA>             PPO Self-Insure~ 05/30/2017\r\n 7 18-416~ 18          <NA>         <NA>          